In [2]:
from langgraph.prebuilt import tools_condition, ToolNode
from IPython.display import Image, display
from langgraph.checkpoint.memory import InMemorySaver
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.types import interrupt, Command
import keys


In [3]:
def process(name : str, age : int) -> str:
    """Use this tool to process a person's admission request.
You must provide:
- name (str): Full name of the applicant
- age (int): Age of the applicant

"""

    if age < 18:
        approval = interrupt(f"Do you want to approve the admission of {name} who is {age} years old? (yes/no):")
        if approval is True:
             return  f"Processed the admission of {name}, who is minor! "
        else:
            return  f"Sorry! Cannot process the admission of {name}, who is a minor! "
    else:
        return  f"Processed the admission of {name}! "

In [4]:
tools = [process]

In [5]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-120b",
    huggingfacehub_api_token=keys.HUGGINGFACE_KEY,
    task="text-generation"   
)

chat = ChatHuggingFace(llm=llm)
llm_with_tools = chat.bind_tools(tools)

# System message
sys_msg = SystemMessage(content="You are a helpful assistant. If the user asks to process admission, you MUST use the process tool.")

In [6]:
# Node
def call_llm(state: MessagesState) -> MessagesState:
    result = llm_with_tools.invoke([sys_msg] + state["messages"])
    return  {"messages": [result]}


In [7]:
# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("call_llm", call_llm)
builder.add_node("tools", ToolNode(tools))
# Define edges: these determine the control flow
builder.add_edge(START, "call_llm")
builder.add_conditional_edges(
    "call_llm",
    tools_condition
)

builder.add_edge("tools", "call_llm")
builder.add_edge("call_llm", END)

In [8]:
memory = InMemorySaver()
graph = builder.compile(checkpointer=memory)

In [9]:
messages = [HumanMessage(
        content="Process admission of Scott Bentton who is 15 years old")]


In [10]:
# Thread
thread = {"configurable": {"thread_id": "1"}}
response = graph.invoke( {"messages" : messages}, thread)

if "__interrupt__" in response:
    question = response['__interrupt__'][0].value
    user_approval = input(question)

    # Check approval
    response = graph.invoke(Command(resume=user_approval.lower() == "yes"), thread)
    print('Result : ', response['messages'][-1].content)
else:
    print("Final Result : ", response['messages'][-1].content)

Result :  Your admission has been processed successfully. 🎉

- **Name:** Scott Bentton  
- **Age:** 15 (minor)

If you need any further assistance or have additional information to provide, just let me know!
